In [1]:
#pip install pulp

In [1]:
import numpy as np
import pandas as pd

In [2]:
from pulp import *

In [3]:
data = pd.read_excel("diet.xls", header = 0)

In [4]:
#eliminating rows with no food data
data = data[:-3]

In [5]:
data.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [6]:
#make list
data2 = data.values.tolist()

In [7]:
#make list of foods
foods = [x[0] for x in data2]

In [20]:
#dictionary of for each nutritional value
price = {x[0]:float(x[1]) for x in data2}
calories = {x[0]:float(x[3]) for x in data2}
cholesterol = {x[0]:float(x[4]) for x in data2}
fat = {x[0]:float(x[5]) for x in data2}
sodium = {x[0]:float(x[6]) for x in data2}
carb = {x[0]:float(x[6]) for x in data2}
fiber = {x[0]:float(x[7]) for x in data2}
protein = {x[0]:float(x[8]) for x in data2}
vita = {x[0]:float(x[9]) for x in data2}
vitc = {x[0]:float(x[10]) for x in data2}
calcium = {x[0]:float(x[11]) for x in data2}
iron = {x[0]:float(x[12]) for x in data2}

In [21]:
#set up the optimaization problem
prob = LpProblem('diet', LpMinimize)

#define vars as dicts
lp_vars = LpVariable.dicts("Foods", foods, 0)

#binary for if food is chose or not
chosenVars = LpVariable.dicts("Chosen",foods,0,1,cat='Binary')

#create objective function
prob += lpSum([price[i] * lp_vars[i] for i in foods]), "Total Cost"


In [ ]:
#create constrains
prob += lpSum([calories[i] * lp_vars[i] for i in foods]) >= 1500, ""
prob += lpSum([calories[i] * lp_vars[i] for i in foods]) <= 2500, ""
prob += lpSum([cholesterol[i] * lp_vars[i] for i in foods]) >= 30, ""
prob += lpSum([cholesterol[i] * lp_vars[i] for i in foods]) <= 240, ""
prob += lpSum([fat[i] * lp_vars[i] for i in foods]) >= 20, ""
prob += lpSum([fat[i] * lp_vars[i] for i in foods]) <= 70, ""
prob += lpSum([sodium[i] * lp_vars[i] for i in foods]) >= 800, ""
prob += lpSum([sodium[i] * lp_vars[i] for i in foods]) <= 2000, ""
prob += lpSum([carb[i] * lp_vars[i] for i in foods]) >= 130, ""
prob += lpSum([carb[i] * lp_vars[i] for i in foods]) <= 450, ""
prob += lpSum([fiber[i] * lp_vars[i] for i in foods]) >= 125, ""
prob += lpSum([fiber[i] * lp_vars[i] for i in foods]) <= 250, ""
prob += lpSum([protein[i] * lp_vars[i] for i in foods]) >= 60, ""
prob += lpSum([protein[i] * lp_vars[i] for i in foods]) <= 100, ""
prob += lpSum([vita[i] * lp_vars[i] for i in foods]) >= 1000, ""
prob += lpSum([vita[i] * lp_vars[i] for i in foods]) <= 10000, ""
prob += lpSum([vitc[i] * lp_vars[i] for i in foods]) >= 400, ""
prob += lpSum([vitc[i] * lp_vars[i] for i in foods]) <= 5000, ""
prob += lpSum([calcium[i] * lp_vars[i] for i in foods]) >= 700, ""
prob += lpSum([calcium[i] * lp_vars[i] for i in foods]) <= 1500, ""
prob += lpSum([iron[i] * lp_vars[i] for i in foods]) >= 10, ""
prob += lpSum([iron[i] * lp_vars[i] for i in foods]) <= 40, ""

for food in foods:
    prob += lp_vars[food] >= 0.1 * foodVars_selected[food]

# If any of a food is eaten, its binary variable must be 1

for food in foods:
    prob += lp

In [27]:
#create constrains
prob += lpSum([calories[i] * lp_vars[i] for i in foods]) >= 1500.00
prob += lpSum([calories[i] * lp_vars[i] for i in foods]) <= 2500.00
prob += lpSum([cholesterol[i] * lp_vars[i] for i in foods]) >= 30.00
prob += lpSum([cholesterol[i] * lp_vars[i] for i in foods]) <= 240.00
prob += lpSum([fat[i] * lp_vars[i] for i in foods]) >= 20.00
prob += lpSum([fat[i] * lp_vars[i] for i in foods]) <= 70.00
prob += lpSum([sodium[i] * lp_vars[i] for i in foods]) >= 800.00
prob += lpSum([sodium[i] * lp_vars[i] for i in foods]) <= 2000.00
prob += lpSum([carb[i] * lp_vars[i] for i in foods]) >= 130.00
prob += lpSum([carb[i] * lp_vars[i] for i in foods]) <= 450.00
prob += lpSum([fiber[i] * lp_vars[i] for i in foods]) >= 125.00
prob += lpSum([fiber[i] * lp_vars[i] for i in foods]) <= 250.00
prob += lpSum([protein[i] * lp_vars[i] for i in foods]) >= 60.00
prob += lpSum([protein[i] * lp_vars[i] for i in foods]) <= 100.00
prob += lpSum([vita[i] * lp_vars[i] for i in foods]) >= 1000.00
prob += lpSum([vita[i] * lp_vars[i] for i in foods]) <= 10000.00
prob += lpSum([vitc[i] * lp_vars[i] for i in foods]) >= 400.00
prob += lpSum([vitc[i] * lp_vars[i] for i in foods]) <= 5000.00
prob += lpSum([calcium[i] * lp_vars[i] for i in foods]) >= 700.00
prob += lpSum([calcium[i] * lp_vars[i] for i in foods]) <= 1500.00
prob += lpSum([iron[i] * lp_vars[i] for i in foods]) >= 10.00
prob += lpSum([iron[i] * lp_vars[i] for i in foods]) <= 40.00

#for i in foods:
#    prob += lp_vars[i]>= chosenVars[i]*0.1
#    prob += lp_vars[i]<= chosenVars[i]*10000

# add contraints to eat at most one of a group of foods    
#prob += chosenVars['Frozen Broccoli'] + chosenVars['Celery, Raw'] <=1

# add contraints that says we require to eat as least 1 from group of food
#prob += (chosenVars['Roasted Chicken'] + chosenVars['Poached Eggs']  
#+ chosenVars['Scrambled Eggs'] + chosenVars['Bologna,Turkey'] 
#+ chosenVars['Frankfurter, Beef'] + chosenVars['Ham,Sliced,Extralean']
#+ chosenVars['Kielbasa,Prk'] + chosenVars['Pizza W/Pepperoni']
#+ chosenVars['Hamburger W/Toppings'] + chosenVars['Hotdog, Plain']
#+ chosenVars['Pork'] + chosenVars['Sardines in Oil']
+ chosenVars['White Tuna in Water'] )>=3

In [28]:
print(prob)

diet:
MINIMIZE
0.23*Foods_2%_Lowfat_Milk + 0.16*Foods_3.3%_Fat,Whole_Milk + 0.24*Foods_Apple,Raw,W_Skin + 0.16*Foods_Apple_Pie + 0.16*Foods_Bagels + 0.15*Foods_Banana + 0.67*Foods_Beanbacn_Soup,W_Watr + 0.15*Foods_Bologna,Turkey + 0.05*Foods_Butter,Regular + 0.31*Foods_Cap'N_Crunch + 0.07*Foods_Carrots,Raw + 0.04*Foods_Celery,_Raw + 0.25*Foods_Cheddar_Cheese + 0.28*Foods_Cheerios + 0.39*Foods_Chicknoodl_Soup + 0.03*Foods_Chocolate_Chip_Cookies + 0.28*Foods_Corn_Flks,_Kellogg'S + 0.39*Foods_Couscous + 0.65*Foods_Crm_Mshrm_Soup,W_Mlk + 0.27*Foods_Frankfurter,_Beef + 0.16*Foods_Frozen_Broccoli + 0.18*Foods_Frozen_Corn + 0.32*Foods_Grapes + 0.33*Foods_Ham,Sliced,Extralean + 0.83*Foods_Hamburger_W_Toppings + 0.31*Foods_Hotdog,_Plain + 0.15*Foods_Kielbasa,Prk + 0.49*Foods_Kiwifruit,Raw,Fresh + 0.02*Foods_Lettuce,Iceberg,Raw + 0.17*Foods_Macaroni,Ckd + 0.52*Foods_Malt_O_Meal,Choc + 0.99*Foods_New_E_Clamchwd,W_Mlk + 0.75*Foods_Neweng_Clamchwd + 0.82*Foods_Oatmeal + 0.09*Foods_Oatmeal_Cookies +

In [29]:
# The problem data is written to an .lp file
#prob.writeLP("Diet.lp")

In [30]:
prob.solve()
print("Status:", LpStatus[prob.status])

Status: Infeasible


In [31]:
for i in prob.variables():
    if i.varValue > 0:
        if str(i).find('Chosen'):
            print(str(i.varValue) + " units of " + str(i))
            
# print the costs of the optimal diet
cost = value(prob.objective)
print("Total cost of food = $", round(cost,2))

41472.801 units of Foods_Peppers,_Sweet,_Raw
830.77278 units of Foods_Tofu
364.88161 units of Foods_White_Tuna_in_Water
Total cost of food = $ 20261.33
